In [1]:
from unsloth import FastLanguageModel
import torch
from langgraph import Estimator  # 랭그래프의 이스티메이터 모듈을 불러옵니다.

max_seq_length = 2048
dtype = None  # None으로 지정할 경우 해당 컴퓨팅 유닛에 알맞은 dtype으로 저장됩니다.
load_in_4bit = True  # 메모리 사용량을 줄이기 위해 4bit 양자화를 사용하실 것을 권장합니다.

# 모델 및 토크나이저 선언
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2-7B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# LoRA Adapter 선언
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None
)

prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["text"]
    outputs = examples["text"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN  # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return {"formatted_text": texts, }
pass

from datasets import load_dataset
dataset = load_dataset("Linq-AI-Research/FinanceRAG", 'FinQABench', split="corpus")
dataset = dataset.map(formatting_prompts_func, batched=True, )
dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="formatted_text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
    ),
)

FastLanguageModel.for_inference(model)

# 랭그래프 이스티메이터 설정 및 평가
def evaluate_with_langgraph(model, tokenizer, inputs):
    # 모델 생성
    outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
    decoded_output = tokenizer.batch_decode(outputs)

    # 평가 기준 설정
    estimator = Estimator(
        metric="f1_score",  # f1_score를 기준으로 사용합니다. 다른 평가 기준도 가능
        threshold=0.9  # 평가 점수가 0.9 이상일 경우에만 통과
    )
    
    # 평가 수행
    score = estimator.evaluate(decoded_output, reference_data="expected_reference_here")  # reference_data에 정답 데이터를 넣어줍니다.
    
    if score >= 0.9:
        print("Evaluation Passed with Score:", score)
        return decoded_output
    else:
        print("Evaluation Failed with Score:", score)
        return None

# 모델 입력 설정
inputs = tokenizer(
    [
        prompt_format.format(
            "다음 중 화폐의 시간가치에 관한 설명으로 옳지 않은 것은 무엇인가? A. 월 복리의 경우, 매월 적용되는 이자율은 연간 명목 이자율을 1/12로 나누어 산출한다. B. 투자 원금 및 기타 조건이 동일할 경우, 단리 방식보다 복리 방식에서 발생하는 이자가 더 크다. C. 일시불로 지급될 금액의 현재 가치는 미래 가치를 일정 기간 동안 할인율을 적용해 산출할 수 있다. D. 1,000,000원을 연 5% 복리로 2년 동안 예치했을 경우, 만기에 받을 세전 이자는 100,000원이다.",
            ""
        )
    ], return_tensors="pt"
).to("cuda")

# 평가 함수 호출
evaluate_with_langgraph(model, tokenizer, inputs)


/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


ImportError: cannot import name 'Estimator' from 'langgraph' (unknown location)